In [1]:
# Use autoplot 207's code for this app
import sys
import datetime
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from shapely.geometry import Point
from pyiem.plot import MapPlot, nwssnow, get_cmap
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
STORM_NUMBER = 9
TITLE = "18-19 January 2024"
SUBTITLE = "8 AM 19 January 2024"
SETPOINT_LOCS = {}
sts = utc(2024, 1, 18, 22)
ets = utc(2024, 1, 19, 18)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
df = df[~df['nwsli'].isin(['DSXI4', 'DMX'])]
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
        """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
        conn,
        params=(sts - datetime.timedelta(days=1), ets)
        )
    print(df[df["state"] == 'IA'])
    mp.plot_values(df.lon.values, df.lat.values, df.magnitude.values, fmt="%.2f", labelbuffer=1, color='purple')

def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2022-2023 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(
        lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df[df["state"] == 'IA'])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1])
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,MI,MQT,26.0,-87.8400,46.5000,POINT (926385.483 237018.628),True,0,True,LSR,22,25
1,MI,MQT,25.3,-87.7500,46.4100,POINT (934713.921 228143.884),True,1,True,LSR,22,25
2,UT,SLC,18.0,-111.5800,40.5900,POINT (-974558.567 -421719.014),True,2,True,LSR,-41,4
3,UT,SLC,17.7,-111.6400,40.5900,POINT (-979566.523 -421007.522),True,3,True,LSR,-41,4
4,IL,DVN,15.4,-90.5500,41.4000,POINT (786446.872 -354994.403),True,4,True,LSR,17,6
...,...,...,...,...,...,...,...,...,...,...,...,...
2902,NV,VEF,0.0,-116.0033,36.2786,POINT (-1428016.472 -832323.901),True,PAHN2,True,COOP,-56,-9
2903,NV,VEF,0.0,-115.1844,36.0467,POINT (-1360208.094 -871379.701),True,VEF,True,COOP,-54,-10
2904,NY,ALY,0.0,-73.4325,42.9361,POINT (2115573.690 120490.856),True,BSKN6,True,COOP,61,22
2905,NY,ALY,0.0,-74.1711,42.7672,POINT (2065075.152 83280.556),True,DLSN6,True,COOP,60,20


In [18]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
        'Z7',
        'Z6',
        296,
    ]
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
        (816, 2),
        (55, 3.9),
        (259, 3),
        ('CIYI4', 0.3),
        ('PGHI4', 2),
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=3)
    # draw_setpoints(mp)

    #plotsqw(mp)
    res = mp.postprocess(filename='240119.png')
    mp.close()

main()

     state  wfo     val       lon      lat                             geo  \
52      IA  DMX  4.5000 -93.74000  41.6400  POINT (519803.357 -353756.944)   
55      IA  DMX  3.9000 -92.84000  41.5800  POINT (594877.975 -354305.899)   
56      IA  DMX  4.5000 -92.34000  40.8600  POINT (643460.691 -430337.388)   
66      IA  DMX  4.1000 -93.66000  41.5300  POINT (527343.383 -365448.323)   
68      IA  DMX  4.0000 -93.59000  41.7400  POINT (531386.873 -341705.462)   
...    ...  ...     ...       ...      ...                             ...   
961     IA  DVN  1.3000 -90.58160  41.6117  POINT (781223.987 -331874.688)   
1000    IA  DVN  0.3000 -91.58330  42.0333  POINT (693857.977 -294211.116)   
1042    IA  ARX  0.0001 -91.79520  43.3043  POINT (662602.115 -155190.678)   
1095    IA  ARX  0.3000 -92.67194  43.0605  POINT (594441.967 -188963.291)   
1096    IA  DVN  0.0000 -90.73500  42.3998  POINT (758952.411 -246112.902)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
5